### Notebook Summary
We will select features by using the addition of the L1 norm to the loss of the LSTM network, thus we will get the more important features by checking the absolute value of the weights.
Finally, I re-ran the network on the reduced features, the accuracy improved in the weekly and monthly forecast, but we will note that this network was not only designed to adjust the weights to the information because of the loss, therefore we will take the results with limited responsibility and later on in the project I will use the features I found here to run the forecasting algorithm

###results
The results of the model are not good, but we will notice that the longer the prediction range is, the model gives less good performances

In [1]:
from google.colab import drive
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import torch

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
Consecutive_stocks =  pd.read_csv('/content/drive/MyDrive/stocks_data/Consecutive_stocks.csv')


In [9]:
loaded_df = pd.read_csv('/content/drive/MyDrive/stocks_data/selected_columns.csv')

# Retrieve lists from loaded DataFrame
columns_to_use_day = loaded_df['columns_to_use_day'].tolist()
columns_to_use_week = loaded_df['columns_to_use_week'].tolist()
columns_to_use_month = loaded_df['columns_to_use_month'].tolist()

In [13]:
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)
from datetime import datetime, timedelta
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support


def xgb_pipline(label_type = 'label_week', df = pd.DataFrame(),columns_to_use = [],start_date = '2021-12-31', alpha = 0.8):

    print(f"Preprocessing... label-{label_type}, split train, val and test, scalling features and resample train to get equally number of samples for each class")
    df = df.copy()
    df.reset_index(drop=True,inplace=True)
    df.rename(columns={label_type: 'y'}, inplace=True)

    # Exclude some columns from the 'df' DataFrame
    if label_type == 'label_week':
        columns_to_exclude = ['label_day', 'y', 'label_month', 'Symbol', 'date','pct_change_week','pct_change_month', 'stock']
    elif label_type == 'label_day':
        columns_to_exclude = ['y', 'label_week', 'label_month', 'Symbol', 'date','pct_change_week','pct_change_month', 'stock']
    elif label_type == 'label_month':
        columns_to_exclude = ['label_day', 'label_week', 'y', 'Symbol', 'date','pct_change_week','pct_change_month', 'stock']
    else:
        columns_to_exclude = ['label_day', 'label_week', 'label_month', 'y', 'Symbol', 'date','pct_change_week','pct_change_month', 'stock']

    all_columns = df.columns
    columns_to_exclude = list(set(all_columns) - set(columns_to_use+['y']))

    df_without_labels = df.drop(columns=columns_to_exclude)
    best_features = columns_to_use
    labels = 'y'

    scaler = StandardScaler()
    df[best_features] = scaler.fit_transform(df[best_features])
    # Split the data into training and validaition  and test sets
    df_model = df[df['y'].isna()==False]
    df_model.reset_index(drop=True,inplace=True)

    df_model['date'] = pd.to_datetime(df_model['date'])

    df_model = df_model[df_model['date']>start_date]
    latest_date = df_model['date'].max()
    first_date = df_model['date'].min()
    num_days = (latest_date - first_date).days
    split_date = first_date + pd.DateOffset(days=int(alpha * num_days))

    End_Train_date = split_date
    Start_test_date = split_date


    train_data = df_model[df_model['date'] < End_Train_date]
    train_data = train_data.sort_values(by=['Symbol','date'])

    test_data  = df_model[df_model['date'] >= Start_test_date]
    class_counts = train_data[labels].value_counts()

    X_train = train_data[best_features]
    y_train = train_data[labels]
    X_test = test_data[best_features]
    y_test = test_data[labels]

    # hanadlling labels imblance by resamleing equally
    min_class_count = train_data['y'].value_counts().min()

    train_data = pd.concat([
        resample(train_data[train_data['y'] == cls].sort_values('date', ascending=False),
                replace=False,
                n_samples=min_class_count,
                random_state=42)
        for cls in train_data['y'].unique()
    ])

    print('Start training')
    model = XGBClassifier(enable_categorical = True,objective='multi:softprob', reg_alpha=0.1,reg_lambda = 0.1)
    model.fit(X_train, y_train.values.ravel())

    class_labels = sorted(df[labels].unique())

    proba_predictions = model.predict_proba(X_test)
    proba_predictions1 = proba_predictions.copy()

    y_pred = proba_predictions1.argmax(axis=1)
    proba_predictions = proba_predictions.max(axis=1)

    # Evaluate the classifier
    print('Evaluating')

    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    report = classification_report(y_test, y_pred)
    unique_classes = list(set(y_test))
    cm = confusion_matrix(y_test, y_pred, labels=unique_classes)

    print(f'Accuracy {accuracy* 100:.2f}')
    print(f'F1-{f1}')
    print(f'report-')
    print(report)
    print('confusion_matrix-')
    print(cm)

    return model,accuracy, f1, report,cm



In [14]:
train_test_ratio=0.8
start_date='2021-12-31'

###Day

In [15]:
model_day,accuracy_day, f1_day, report_day,cm_day  = xgb_pipline(label_type = 'label_day', df = Consecutive_stocks, columns_to_use = columns_to_use_day,start_date = start_date, alpha = train_test_ratio)


Preprocessing... label-label_day, split train, val and test, scalling features and resample train to get equally number of samples for each class


<ipython-input-13-74a18241a0ec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['date'] = pd.to_datetime(df_model['date'])


Start training
Evaluating
Accuracy 36.05
F1-0.31234283348407316
report-
              precision    recall  f1-score   support

         0.0       0.11      0.03      0.05      2934
         1.0       0.11      0.02      0.03     14619
         2.0       0.13      0.02      0.03     39282
         3.0       0.35      0.30      0.32    159159
         4.0       0.39      0.64      0.48    171547
         5.0       0.13      0.03      0.05     36074
         6.0       0.08      0.03      0.05     15637
         7.0       0.09      0.01      0.02      4735

    accuracy                           0.36    443987
   macro avg       0.17      0.14      0.13    443987
weighted avg       0.30      0.36      0.31    443987

confusion_matrix-
[[    83    143    123    705   1576    116    137     51]
 [    62    303    467   3705   8923    461    606     92]
 [    62    334    720  11085  25165    955    883     78]
 [   168    494   1308  47752 105539   2327   1470    101]
 [   177    573   1546 

###Week

In [16]:
model_week,accuracy_week, f1_week, report_week, cm_week = xgb_pipline(label_type = 'label_week', df = Consecutive_stocks, columns_to_use = columns_to_use_week,start_date = start_date, alpha = train_test_ratio)


Preprocessing... label-label_week, split train, val and test, scalling features and resample train to get equally number of samples for each class


<ipython-input-13-74a18241a0ec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['date'] = pd.to_datetime(df_model['date'])


Start training
Evaluating
Accuracy 22.47
F1-0.19860980733459133
report-
              precision    recall  f1-score   support

         0.0       0.17      0.22      0.19     22285
         1.0       0.16      0.24      0.19     47412
         2.0       0.20      0.11      0.14     58020
         3.0       0.25      0.35      0.29     89802
         4.0       0.27      0.41      0.32     96108
         5.0       0.12      0.02      0.03     54450
         6.0       0.15      0.05      0.07     44959
         7.0       0.21      0.12      0.16     26754

    accuracy                           0.22    439790
   macro avg       0.19      0.19      0.17    439790
weighted avg       0.20      0.22      0.20    439790

confusion_matrix-
[[ 4811  6104  1275  2581  2993   507  1302  2712]
 [ 4298 11249  4891 11480 10177   889  2044  2384]
 [ 2945  9555  6126 18708 16438  1096  1620  1532]
 [ 3352 11207  7116 31275 31950  1619  1840  1443]
 [ 3504 11638  5560 31306 38940  1526  2070  1564]
 [ 2

###Month

In [17]:
model_month,accuracy_month, f1_month, report_month, cm_month =  xgb_pipline(label_type = 'label_month', df = Consecutive_stocks, columns_to_use = columns_to_use_month,start_date = start_date, alpha = train_test_ratio)


Preprocessing... label-label_month, split train, val and test, scalling features and resample train to get equally number of samples for each class


<ipython-input-13-74a18241a0ec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['date'] = pd.to_datetime(df_model['date'])


Start training
Evaluating
Accuracy 19.51
F1-0.17318391734898014
report-
              precision    recall  f1-score   support

         0.0       0.28      0.40      0.33     93997
         1.0       0.09      0.09      0.09     60496
         2.0       0.07      0.01      0.02     35736
         3.0       0.16      0.07      0.09     36239
         4.0       0.20      0.12      0.15     37715
         5.0       0.08      0.02      0.03     29267
         6.0       0.09      0.11      0.10     46637
         7.0       0.21      0.31      0.25     91511

    accuracy                           0.20    431598
   macro avg       0.15      0.14      0.13    431598
weighted avg       0.17      0.20      0.17    431598

confusion_matrix-
[[37848  3451   182   791  1800   939  9074 39912]
 [15486  5325   465  1863  3921  1615 12631 19190]
 [ 8034  4108   383  1836  2844  1234  7668  9629]
 [ 8040  5086   426  2433  3345  1042  6787  9080]
 [ 8609  6155   574  2536  4396   860  5612  8973]
 [ 7